# Matrix Factorisation

We look at three matrix factorisation methods: Non-negative Matrix Factorisation (NMF), Singular Value Decomposition (SVD) and Alternating Least Squares (alsMF).

Matrix factorization is a class of collaborative filtering algorithms used in recommender systems. Matrix factorization algorithms work by decomposing the user-item interaction matrix. 

***
# Sandbox

Here we will test out the workings of matrix factorisation collaborative filtering. Specifically, we will be conducting non-negative matrix factorisation (NMF) and alternating least squares (ALS) matrix factorisation. We will be using the sample data created. The steps are as follows:

1. Have User Item matrix
2. Hide some ratings to simulate a test set
3. Factorise the matrix - *either NMF or ALS or even SVD*
4. Predict the hidden ratings - fill in missing values with predicted ratings
6. Take the predicted ratings and compare them to the hidden ratings
7. Calculate MAE, RMSE, MSE
8. Binarise the ratings
9. Calculate classification metrics

## Matrix Factorisation w/SGD

- Matrix Factorization is a general technique used in collaborative filtering and other applications where a matrix is decomposed into the product of two lower-rank matrices.
- Stochastic Gradient Descent is an optimization algorithm commonly used to minimize the error in the factorization process.

- The key idea is to iteratively update the elements of the factorized matrices using the gradient of the error with respect to the elements.

**Algorithm Process**:

1. ***Convert the data into a matrix:*** Represent users, items, and ratings in a matrix where rows correspond to users, columns correspond to items, and values represent ratings.

2. ***Hide some ratings for testing:*** Randomly select a subset of ratings to be used as a test set to evaluate the performance of your collaborative filtering algorithm.

3. ***Decompose the matrix using SGD:*** Decompose the original matrix into two matrices - one representing users and latent features (user matrix) and the other representing items and latent features (item matrix).

4. ***Reconstruct the original matrix:*** Take the dot product of the user and item matrices obtained from step 3 to reconstruct the original matrix.

5. ***Make predictions using the reconstructed matrix:*** Use the reconstructed matrix to predict the ratings for the items that were hidden in the test set. These predictions will be your estimated ratings.

6. ***Evaluate the performance of the algorithm:*** Compare the predicted ratings to the actual ratings in the test set to evaluate the accuracy and effectiveness of your collaborative filtering algorithm. Common evaluation metrics include Mean Squared Error (MSE), Root Mean Squared Error (RMSE), or other relevant metrics.

In [19]:
%reset -f

# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
x = pd.read_csv(r"C:\Users\e1002902\Documents\GitHub Repository\Masters-Dissertation\Code\temp_data.csv", index_col=0)
x

,book1,book2,book3,book4,book5,book6,book7,book8,book9,book10
user1,0,0,2,5,4,3,4,4,4,4
user2,4,0,3,5,0,0,0,0,0,4
user3,0,3,4,4,0,2,0,0,0,0
user4,0,0,3,5,4,0,0,0,0,0
user5,3,4,0,4,4,0,5,5,5,5
user6,4,5,0,0,0,0,4,2,2,0
user7,2,2,0,0,0,0,5,3,3,3
user8,0,5,4,0,4,3,0,0,0,0
user9,0,5,4,0,5,2,0,2,2,0
user10,0,0,0,0,5,0,4,4,4,4


In [21]:
# create a copy of the original matrix to store hidden ratings
x_hidden = x.copy()
indices_tracker = []

# identifies rated books and randomly selects 2 books to hide ratings for each user
np.random.seed(10)  # You can use any integer value as the seed
for user_id in range(x_hidden.shape[0]):
    rated_books = np.where(x_hidden.iloc[user_id, :] > 0)[0]
    print("User:", user_id)
    print("Indices of Rated Books:", rated_books)
    hidden_indices = np.random.choice(rated_books, min(2, len(rated_books)), replace=False)
    indices_tracker.append(hidden_indices)
    print("Indices to Hide:", hidden_indices, "\n")
    x_hidden.iloc[user_id, hidden_indices] = 0


User: 0
Indices of Rated Books: [2 3 4 5 6 7 8 9]
Indices to Hide: [4 5] 

User: 1
Indices of Rated Books: [0 2 3 9]
Indices to Hide: [3 9] 

User: 2
Indices of Rated Books: [1 2 3 5]
Indices to Hide: [5 1] 

User: 3
Indices of Rated Books: [2 3 4]
Indices to Hide: [4 2] 

User: 4
Indices of Rated Books: [0 1 3 4 6 7 8 9]
Indices to Hide: [9 1] 

User: 5
Indices of Rated Books: [0 1 6 7 8]
Indices to Hide: [8 1] 

User: 6
Indices of Rated Books: [0 1 6 7 8 9]
Indices to Hide: [8 9] 

User: 7
Indices of Rated Books: [1 2 4 5]
Indices to Hide: [1 5] 

User: 8
Indices of Rated Books: [1 2 4 5 7 8]
Indices to Hide: [1 7] 

User: 9
Indices of Rated Books: [4 6 7 8 9]
Indices to Hide: [6 4] 

User: 10
Indices of Rated Books: [0 1 2 4 6 7 8]
Indices to Hide: [2 0] 

User: 11
Indices of Rated Books: [0 1 2 4 5 6 7 8]
Indices to Hide: [1 7] 



In [22]:
# check tracker - all hidden ratings 
indices_tracker = pd.DataFrame(indices_tracker).to_numpy()
print("Indices of Ratings per user \n", indices_tracker)

# flattened
indices_tracker_flat = indices_tracker.flatten()
print("Indices of Ratings per User joined", indices_tracker_flat)


Indices of Ratings per user 
 [[4 5]
 [3 9]
 [5 1]
 [4 2]
 [9 1]
 [8 1]
 [8 9]
 [1 5]
 [1 7]
 [6 4]
 [2 0]
 [1 7]]
Indices of Ratings per User joined [4 5 3 9 5 1 4 2 9 1 8 1 8 9 1 5 1 7 6 4 2 0 1 7]


In [23]:
# see updated matrix with hidden ratings
print("Updated Matrix with Hidden Ratings")
display(x_hidden)

# see original matrix
print("Original Matrix")
display(x)

Updated Matrix with Hidden Ratings


,book1,book2,book3,book4,book5,book6,book7,book8,book9,book10
user1,0,0,2,5,0,0,4,4,4,4
user2,4,0,3,0,0,0,0,0,0,0
user3,0,0,4,4,0,0,0,0,0,0
user4,0,0,0,5,0,0,0,0,0,0
user5,3,0,0,4,4,0,5,5,5,0
user6,4,0,0,0,0,0,4,2,0,0
user7,2,2,0,0,0,0,5,3,0,0
user8,0,0,4,0,4,0,0,0,0,0
user9,0,0,4,0,5,2,0,0,2,0
user10,0,0,0,0,0,0,0,4,4,4


Original Matrix


,book1,book2,book3,book4,book5,book6,book7,book8,book9,book10
user1,0,0,2,5,4,3,4,4,4,4
user2,4,0,3,5,0,0,0,0,0,4
user3,0,3,4,4,0,2,0,0,0,0
user4,0,0,3,5,4,0,0,0,0,0
user5,3,4,0,4,4,0,5,5,5,5
user6,4,5,0,0,0,0,4,2,2,0
user7,2,2,0,0,0,0,5,3,3,3
user8,0,5,4,0,4,3,0,0,0,0
user9,0,5,4,0,5,2,0,2,2,0
user10,0,0,0,0,5,0,4,4,4,4


In [35]:
# matrix factorization using stochastic gradient descent
def matrix_factorization_sgd(R, K, steps=5000, alpha=0.0002, beta=0.02):
    # R = user-item ratings matrix
    # K = number of latent features
    # steps = number of iterations
    # alpha = learning rate
    # beta = regularization parameter

    # Initialize user and item latent feature matrices
    N, M = R.shape
    P = np.random.rand(N, K)
    Q = np.random.rand(M, K)
    Q = Q.T
    
    # apply stochastic gradient descent to update P and Q
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P, Q)
        e = 0
        # apply regularization to prevent overfitting
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        # set threshold for error rate
        if e < 0.001:
            break
    return P, Q.T

# apply matrix factorization using stochastic gradient descent
nP, nQ = matrix_factorization_sgd(R=x_hidden.values, K=2)
nR = np.dot(nP, nQ.T)

# view reconstructed matrix
print("Reconstructed Matrix")
nR = pd.DataFrame(nR, index=x_hidden.index, columns=x_hidden.columns)
nR

Reconstructed Matrix


,book1,book2,book3,book4,book5,book6,book7,book8,book9,book10
user1,3.224972,1.698436,3.380388,4.126833,4.153397,2.460501,4.380824,3.679860,3.674275,3.859222
user2,3.985326,1.226839,2.987952,3.196596,2.791390,2.626943,3.392883,3.555286,3.396073,3.632177
user3,4.299593,1.674545,3.702271,4.214642,3.953775,3.000581,4.473732,4.234956,4.124487,4.376170
user4,3.607138,2.071729,4.015606,4.991314,5.107436,2.833679,5.298600,4.311656,4.335454,4.540828
user5,3.772023,1.857752,3.778138,4.545785,4.512163,2.816781,4.825494,4.157542,4.128515,4.345950
user6,3.262874,1.309905,2.862946,3.283800,3.105486,2.295641,3.485694,3.258339,3.181340,3.371998
user7,1.857698,1.980481,3.314084,4.564329,5.083007,1.892715,4.845766,3.259912,3.431724,3.529583
user8,4.347409,1.739023,3.805989,4.361592,4.120857,3.055703,4.629747,4.334225,4.230537,4.484622
user9,2.074136,1.797601,3.136034,4.193381,4.564722,1.917021,4.451833,3.169112,3.288699,3.401540
user10,3.996602,1.590418,3.487581,3.991576,3.766109,2.805201,4.236976,3.975065,3.878298,4.111951


In [36]:
# now evaluate how good the predictions are vs the hidden ratings
# step 1: identify the hidden ratings indices
# step 2: extract hidden ratings indices and corresponding predicted ratings indices
# step 3: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values)
# step 4:  binarise to get classification metrics

# step 1: identify the hidden ratings indices = indices_tracker and get the hidden ratings ==========================================================================
hidden_ratings_ind = indices_tracker.copy()

# Loop through users to append hidden ratings
hidden_ratings_arrays = []

# Loop through users to append hidden ratings arrays
for user in range(x.shape[0]):
    user_hidden_ratings = x.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    hidden_ratings_arrays.append(user_hidden_ratings)


hidden_ratings_array = pd.DataFrame(hidden_ratings_arrays).to_numpy().flatten()
print("Hidden Ratings:", hidden_ratings_array)


Hidden Ratings: [4 3 5 4 2 3 4 3 5 4 2 5 3 3 5 3 5 2 4 5 2 4 5 3]


In [37]:
# step 2: extract corresponding predicted ratings indices ==========================================================================

# Create an empty list to store predicted ratings arrays
predicted_ratings_arrays = []

# Loop through users to append predicted ratings arrays
for user in range(nR.shape[0]):
    user_predicted_ratings = nR.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    predicted_ratings_arrays.append(user_predicted_ratings)

predicted_ratings_array = pd.DataFrame(predicted_ratings_arrays).to_numpy().flatten()
print("Corresponding Predicted Ratings:", predicted_ratings_array)


Corresponding Predicted Ratings: [4.15339727 2.46050078 3.1965964  3.63217674 3.00058092 1.67454501
 5.10743615 4.0156065  4.34594978 1.85775248 3.18134028 1.30990463
 3.43172431 3.52958302 1.73902281 3.05570287 1.79760063 3.16911205
 4.23697599 3.76610891 4.35679929 4.88445879 0.94545239 3.72544073]


In [38]:
# step 3: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values) ==========================================================================

from sklearn.metrics import mean_absolute_error, mean_squared_error

# calculate MAE, MSE and RMSE
print("Using sklearn")
mae = mean_absolute_error(hidden_ratings_array, predicted_ratings_array)
mse = mean_squared_error(hidden_ratings_array, predicted_ratings_array)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


# Manually
print("\n\nManually")
mae = np.mean(np.abs(hidden_ratings_array - predicted_ratings_array)) # Calculate Mean Absolute Error (MAE)
mse = np.mean((hidden_ratings_array - predicted_ratings_array) ** 2) # Calculate Mean Squared Error (MSE)
rmse = np.sqrt(mse) # Calculate Root Mean Squared Error (RMSE)


print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Using sklearn
Mean Absolute Error (MAE): 1.3801061499799818
Mean Squared Error (MSE): 3.1814386429508974
Root Mean Squared Error (RMSE): 1.7836587798541788


Manually
Mean Absolute Error (MAE): 1.3801061499799818
Mean Squared Error (MSE): 3.1814386429508974
Root Mean Squared Error (RMSE): 1.7836587798541788


In [39]:
# step 4: calculate Classification Metrics (take the hidden ratings and the predicted ratings and binarise them) ==========================================================================

# Binarise the hidden ratings and predicted ratings
threshold = 3.5
binary_prediction_ratings = (predicted_ratings_array >= threshold).astype(int) 
print(f"If predicted rating is greater than or equal to {threshold}, then 1, else 0\n")
print("Predicted Ratings:", predicted_ratings_array)
print("Binary Predictions:", binary_prediction_ratings)
binary_hidden_ratings = (hidden_ratings_array >= threshold).astype(int)
print("\n")

print("Hidden Ratings:", hidden_ratings_array)
print("Binary Hidden Ratings:", binary_hidden_ratings)

If predicted rating is greater than or equal to 3.5, then 1, else 0

Predicted Ratings: [4.15339727 2.46050078 3.1965964  3.63217674 3.00058092 1.67454501
 5.10743615 4.0156065  4.34594978 1.85775248 3.18134028 1.30990463
 3.43172431 3.52958302 1.73902281 3.05570287 1.79760063 3.16911205
 4.23697599 3.76610891 4.35679929 4.88445879 0.94545239 3.72544073]
Binary Predictions: [1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 0 1]


Hidden Ratings: [4 3 5 4 2 3 4 3 5 4 2 5 3 3 5 3 5 2 4 5 2 4 5 3]
Binary Hidden Ratings: [1 0 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1 0 1 1 0]


In [40]:
# calculate accuracy using sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# calculate accuracy using sklearn
print("Using sklearn")
accuracy = accuracy_score(binary_hidden_ratings, binary_prediction_ratings)
precision = precision_score(binary_hidden_ratings, binary_prediction_ratings)
recall = recall_score(binary_hidden_ratings, binary_prediction_ratings)
f1 = f1_score(binary_hidden_ratings, binary_prediction_ratings)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# calculate accuracy manually
print("\n\nManually")
true_positives = np.sum((binary_hidden_ratings == 1) & (binary_prediction_ratings == 1))
true_negatives = np.sum((binary_hidden_ratings == 0) & (binary_prediction_ratings == 0))
false_positives = np.sum((binary_hidden_ratings == 0) & (binary_prediction_ratings == 1))
false_negatives = np.sum((binary_hidden_ratings == 1) & (binary_prediction_ratings == 0))

accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1 = 2 * precision * recall / (precision + recall)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Using sklearn
Accuracy: 0.5833333333333334
Precision: 0.6363636363636364
Recall: 0.5384615384615384
F1 Score: 0.5833333333333334


Manually
Accuracy: 0.5833333333333334
Precision: 0.6363636363636364
Recall: 0.5384615384615384
F1 Score: 0.5833333333333334


### Alternating Least Squares (ALS)

The main difference is in the optimization method used to decompose the matrix. Instead of using NMF, you use the ALS algorithm.

**Algorithm Process**:

***Convert the data into a matrix:*** Same as in NMF, represent users, items, and ratings in a matrix.

***Hide some ratings for testing:*** Randomly select a subset of ratings to be used as a test set.

***Decompose the matrix using ALS:*** Utilize an ALS algorithm to iteratively update the user and item matrices until convergence. You may need to define hyperparameters such as the number of latent features and regularization terms.

***Reconstruct the original matrix:*** Combine the user and item matrices obtained from the ALS optimization to reconstruct the original matrix.

***Make predictions using the reconstructed matrix:*** Use the reconstructed matrix to predict the ratings for the items that were hidden in the test set.

***Evaluate the performance of the algorithm:*** Compare the predicted ratings to the actual ratings in the test set to evaluate the accuracy and effectiveness of your collaborative filtering algorithm. Use appropriate evaluation metrics such as MSE, RMSE, etc.


In [60]:
%reset -f

# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
x = pd.read_csv(r"C:\Users\e1002902\Documents\GitHub Repository\Masters-Dissertation\Code\temp_data.csv", index_col=0)

# create a copy of the original matrix to store hidden ratings
x_hidden = x.copy()
indices_tracker = []

# identifies rated books and randomly selects 2 books to hide ratings for each user
np.random.seed(10)  # You can use any integer value as the seed
for user_id in range(x_hidden.shape[0]):
    rated_books = np.where(x_hidden.iloc[user_id, :] > 0)[0]
    hidden_indices = np.random.choice(rated_books, min(2, len(rated_books)), replace=False)
    indices_tracker.append(hidden_indices)
    x_hidden.iloc[user_id, hidden_indices] = 0

# check tracker - all hidden ratings 
indices_tracker = pd.DataFrame(indices_tracker).to_numpy()

# flattened
indices_tracker_flat = indices_tracker.flatten()

In [65]:
def matrix_factorization_als(R, K, steps=5000, alpha=0.0001, reg_param=0.001):
    N, M = R.shape
    P = np.random.rand(N, K) * 0.01
    Q = np.random.rand(M, K) * 0.01

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[j, :].T)
                    P_norm = np.linalg.norm(P[i, :], 2)
                    Q_norm = np.linalg.norm(Q[j, :], 2)
                    P[i, :] = P[i, :] + alpha * (2 * eij * Q[j, :] - reg_param * P_norm)
                    Q[j, :] = Q[j, :] + alpha * (2 * eij * P[i, :] - reg_param * Q_norm)

        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i, :], Q[j, :].T), 2)
        if e < 0.001:
            break

    return P, Q


# apply matrix factorization using alternating least squares
nP, nQ = matrix_factorization_als(R = x_hidden.values, K=2)
nR = np.dot(nP, nQ.T)

# view reconstructed matrix
print("Reconstructed Matrix")
nR = pd.DataFrame(nR, index=x_hidden.index, columns=x_hidden.columns)
nR

Reconstructed Matrix


,book1,book2,book3,book4,book5,book6,book7,book8,book9,book10
user1,3.745105,1.786897,3.302105,4.228972,3.759945,2.591919,4.325972,3.797515,3.730854,3.813295
user2,3.715455,1.742278,3.285085,4.108012,3.695029,2.540137,4.215475,3.681452,3.587693,3.705626
user3,3.931880,1.843308,3.476578,4.345988,3.909736,2.687630,4.459880,3.894603,3.794967,3.920314
user4,4.348004,2.051932,3.840462,4.844811,4.339134,2.985963,4.965766,4.344997,4.247092,4.369642
user5,4.136711,1.964370,3.650195,4.644262,4.142290,2.853326,4.754861,4.168146,4.086016,4.188197
user6,3.232091,1.509637,2.859500,3.556413,3.207425,2.203543,3.652100,3.185637,3.098655,3.208338
user7,3.298082,1.562067,2.911414,3.691058,3.297836,2.270701,3.780734,3.311662,3.242493,3.328785
user8,4.223466,1.978497,3.734851,4.663950,4.197938,2.885395,4.786845,4.179164,4.070771,4.207204
user9,3.764182,1.744780,3.334260,4.103472,3.720018,2.552576,4.219845,3.672307,3.558870,3.702487
user10,4.048184,1.910024,3.575764,4.509542,4.039447,2.779638,4.622310,4.044213,3.952684,4.067276


In [73]:
# now evaluate how good the predictions are vs the hidden ratings
# step 1: identify the hidden ratings indices
# step 2: extract hidden ratings indices and corresponding predicted ratings indices
# step 3: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values)
# step 4:  binarise to get classification metrics

# step 1: identify the hidden ratings indices = indices_tracker and get the hidden ratings ==========================================================================
hidden_ratings_ind = indices_tracker.copy()

# Loop through users to append hidden ratings
hidden_ratings_arrays = []

# Loop through users to append hidden ratings arrays
for user in range(x.shape[0]):
    user_hidden_ratings = x.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    hidden_ratings_arrays.append(user_hidden_ratings)


hidden_ratings_array = pd.DataFrame(hidden_ratings_arrays).to_numpy().flatten()


# step 2: extract corresponding predicted ratings indices ==========================================================================

# Create an empty list to store predicted ratings arrays
predicted_ratings_arrays = []

# Loop through users to append predicted ratings arrays
for user in range(nR.shape[0]):
    user_predicted_ratings = nR.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    predicted_ratings_arrays.append(user_predicted_ratings)

predicted_ratings_array = pd.DataFrame(predicted_ratings_arrays).to_numpy().flatten()


# step 3: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values) ==========================================================================

from sklearn.metrics import mean_absolute_error, mean_squared_error

# calculate MAE, MSE and RMSE
print("\nRATINGS PREDICTION PROBLEM\nUsing sklearn")
mae = mean_absolute_error(hidden_ratings_array, predicted_ratings_array)
mse = mean_squared_error(hidden_ratings_array, predicted_ratings_array)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


# Manually
print("\nManually")
mae = np.mean(np.abs(hidden_ratings_array - predicted_ratings_array)) # Calculate Mean Absolute Error (MAE)
mse = np.mean((hidden_ratings_array - predicted_ratings_array) ** 2) # Calculate Mean Squared Error (MSE)
rmse = np.sqrt(mse) # Calculate Root Mean Squared Error (RMSE)


print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

# step 4: calculate Classification Metrics (take the hidden ratings and the predicted ratings and binarise them) ==========================================================================

# Binarise the hidden ratings and predicted ratings
threshold = 3.5
binary_prediction_ratings = (predicted_ratings_array >= threshold).astype(int) 
binary_hidden_ratings = (hidden_ratings_array >= threshold).astype(int)
print("\n\nBINARISED or CLASSIFICATION PROBLEM")

# calculate accuracy using sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# calculate accuracy using sklearn
print("Using sklearn")
accuracy = accuracy_score(binary_hidden_ratings, binary_prediction_ratings)
precision = precision_score(binary_hidden_ratings, binary_prediction_ratings)
recall = recall_score(binary_hidden_ratings, binary_prediction_ratings)
f1 = f1_score(binary_hidden_ratings, binary_prediction_ratings)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# calculate accuracy manually
print("\nManually")
true_positives = np.sum((binary_hidden_ratings == 1) & (binary_prediction_ratings == 1))
true_negatives = np.sum((binary_hidden_ratings == 0) & (binary_prediction_ratings == 0))
false_positives = np.sum((binary_hidden_ratings == 0) & (binary_prediction_ratings == 1))
false_negatives = np.sum((binary_hidden_ratings == 1) & (binary_prediction_ratings == 0))

accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1 = 2 * precision * recall / (precision + recall)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")



RATINGS PREDICTION PROBLEM
Using sklearn
Mean Absolute Error (MAE): 1.2213713115367435
Mean Squared Error (MSE): 2.6568159400546336
Root Mean Squared Error (RMSE): 1.629974214536731

Manually
Mean Absolute Error (MAE): 1.2213713115367435
Mean Squared Error (MSE): 2.6568159400546336
Root Mean Squared Error (RMSE): 1.629974214536731


BINARISED or CLASSIFICATION PROBLEM
Using sklearn
Accuracy: 0.6666666666666666
Precision: 0.7272727272727273
Recall: 0.6153846153846154
F1 Score: 0.6666666666666667

Manually
Accuracy: 0.6666666666666666
Precision: 0.7272727272727273
Recall: 0.6153846153846154
F1 Score: 0.6666666666666667
